In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/gender_submission.csv
/kaggle/input/titanic/test.csv


In [15]:
#read data
def read_data():
    tmp_dict = {}
    for dirname, _, filenames in os.walk('/kaggle/input'):
        for filename in filenames:
            tmp_dict[filename.split('.csv')[0]]= pd.read_csv(os.path.join(dirname, filename))
    return tmp_dict['train'], tmp_dict['test'], tmp_dict['gender_submission']

In [18]:
train_data, test_data, gender_submission = read_data()

In [19]:
# import pandas_profiling as pp
# report = pp.ProfileReport(train_data)
# report

In [20]:
def data_cleansing(data_df):
    """
    Carbin has too many miss values. Remove it.
    Ticket number are meaningless. Remove it.
    Ｆill mode value in these features:
    'Pclass', 'Sex', 'Age', 'SibSp',
     'Parch', 'Fare', 'Embarked'
     Extact family name from 'Name'
    """
    FILL_MODE_FEATURE = [
     'Pclass', 'Sex', 'Age', 'SibSp',
     'Parch', 'Fare', 'Embarked'  
    ]
    
    label = data_df["Survived"]
    
    data_df.drop('Cabin', axis=1, inplace=True)
    data_df.drop('Ticket', axis=1, inplace=True)
    data_df.drop('Survived', axis=1, inplace=True)
    
    
    for feature in FILL_MODE_FEATURE:
        data_df[feature]=data_df[feature].fillna(train_data[feature][0])
        
    data_df['family_name'] = [name.split(',')[0] for name in data_df['Name']]
    
    FILL_MODE_FEATURE.extend(['family_name','Name'])
    
    onehotencode = pd.get_dummies(data_df[FILL_MODE_FEATURE])
    
    data_df = pd.concat([train_data,onehotencode], axis=1)
    
    #Remove the feature that applied onehotencode
    for feature in FILL_MODE_FEATURE:
        data_df.drop(feature, axis=1, inplace=True)
        
    return data_df,label

In [21]:
train_X,train_Y= data_cleansing(train_data)

In [17]:
train_data.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [22]:
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
# X, y = load_iris(return_X_y=True)
clf = LogisticRegression(random_state=0).fit(train_X, train_Y)

/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [24]:
predicted = clf.predict(train_X)

In [25]:
from sklearn.metrics import classification_report

In [32]:
result = classification_report(train_Y,predicted,output_dict=True)['macro avg']

KeyError: 'marco avg'

In [33]:
pd.DataFrame([result])

,0,1,accuracy,macro avg,weighted avg
0,"{'precision': 0.8637873754152824, 'recall': 0....","{'precision': 0.8996539792387543, 'recall': 0....",0.875421,"{'precision': 0.8817206773270183, 'recall': 0....","{'precision': 0.8775543546606555, 'recall': 0...."
